# Setup on Colab

In [ ]:
!git clone https://github.com/chris838/alpha-zero-general.git

Cloning into 'alpha-zero-general'...
remote: Enumerating objects: 1179, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 1179 (delta 18), reused 42 (delta 16), pack-reused 1135
Receiving objects: 100% (1179/1179), 427.27 MiB | 15.61 MiB/s, done.
Resolving deltas: 100% (642/642), done.
Checking out files: 100% (129/129), done.


In [ ]:
%cd '/content/alpha-zero-general'

/content/alpha-zero-general


In [ ]:
# !pip install -r docker/requirements.txt
!pip install coloredlogs ipympl  # numpy imgaug folium torchvision tqdm scipy tqdm pandas scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Wed Jul 27 12:43:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from pathlib import Path
checkpoint_folder = Path('/content/gdrive/MyDrive/colab/alpha-zero-general/temp')
load_folder = Path('/content/gdrive/MyDrive/colab/alpha-zero-general/pretrained_models/santorini/keras/5x5')

# Setup locally

In [ ]:
from pathlib import Path
checkpoint_folder = Path('temp')
load_folder = Path('pretrained_models/santorini/keras/5x5')

# Imports

In [ ]:
from datetime import datetime
import shutil
import logging
import coloredlogs
from Coach import Coach
from utils import dotdict
from santorini.keras.NNet import NNetWrapper
from santorini.SantoriniGame import SantoriniGame

In [ ]:
import Arena
from MCTS import MCTS

from santorini.SantoriniPlayers import (
    RandomPlayer,
    HumanSantoriniPlayer,
    GreedySantoriniPlayer,
)

import numpy as np
from utils import *

# Train AlphaZero

In [ ]:
log = logging.getLogger(__name__)
coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

In [ ]:
checkpoint_folder.mkdir(parents=True, exist_ok=True)
load_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'checkpoint': str(checkpoint_folder),
    'numItersForTrainExamplesHistory': 20,
})

In [ ]:
game = SantoriniGame(5)
nnet = NNetWrapper(game)

# Set very low iterations to let this notebook run in its entirety.
# In reality, training a model, even as simple as the one for Dots and Boxes, can take several hours or days.
args['numIters'] = 100
args['numEps'] = 200
args['arenaCompare'] = 30

coach = Coach(game, nnet, args)

In [ ]:
# Check if there are any checkpoints to load
if list(load_folder.glob('*.h5')):
  
  print ("Resuming training from saved model and samples")

  # Find the latest model/examples
  last_checkpoint_name = sorted(load_folder.glob('*.examples'))[-1].stem
  model_file = last_checkpoint_name # .pth.tar substituted for h5 in the load method
  checkpoint_file = last_checkpoint_name # 'examples' suffix appended in the load method

  # Load model weights
  print (f"Loading model weights from {model_file.replace('.pth.tar', '.h5')}")
  nnet.load_checkpoint(folder=str(load_folder), filename=model_file)

  # Load training examples
  print (f"Loading training samples from {checkpoint_file}.examples")
  args['load_model'] = True
  args['load_folder_file'] = (str(load_folder), checkpoint_file)
  coach.loadTrainExamples()

  # Since we've loaded both model and examples, we want to start with self
  # play, so unset the flag that is set by loadTrainExamples()
  coach.skipFirstSelfPlay = False

else:
  args['load_model'] = False
  print ("No progress to load, training from scratch")

2022-07-27 12:51:37 e4b1754583f3 Coach[88] INFO File with trainExamples found. Loading it...


Resuming training from saved model and samples
Loading model weights from 2022-07-27_12-49-53.h5
Loading training samples from 2022-07-27_12-49-53.pth.tar.examples


2022-07-27 12:52:22 e4b1754583f3 Coach[88] INFO Loading done!


In [ ]:
%tensorflow_version 2.x

# Learn
%time coach.learn()

2022-07-25 16:59:09 72167a8923bc Coach[90] INFO Starting Iter #1 ...
Self Play: 100%|██████████| 200/200 [1:01:53<00:00, 18.57s/it]


Checkpoint Directory exists! 
Epoch 1/10
3830/3830 [==============================] - 52s 13ms/step - loss: 3.0128 - pi_loss: 2.4263 - v_loss: 0.5866
Epoch 2/10
3830/3830 [==============================] - 49s 13ms/step - loss: 2.6644 - pi_loss: 2.2018 - v_loss: 0.4626
Epoch 3/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.4667 - pi_loss: 2.0670 - v_loss: 0.3996
Epoch 4/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.3288 - pi_loss: 1.9682 - v_loss: 0.3606
Epoch 5/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.2223 - pi_loss: 1.8906 - v_loss: 0.3317
Epoch 6/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.1381 - pi_loss: 1.8300 - v_loss: 0.3080
Epoch 7/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.0773 - pi_loss: 1.7852 - v_loss: 0.2921
Epoch 8/10
3830/3830 [==============================] - 48s 13ms/step - loss: 2.0228 - pi_loss: 1.7449 - v_loss: 0.2780
Epoch 9/10

2022-07-25 18:10:45 72167a8923bc Coach[90] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (1):  47%|████▋     | 7/15 [01:33<01:35, 11.90s/it]2022-07-25 18:12:33 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:12:34 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:12:34 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2):  40%|████      | 6/15 [01:47<02:40, 17.86s/it]2022-07-25 18:16:22 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:16:22 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:16:22 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2): 100%|██████████| 15/15 [04:44<00:00, 19.00s/it]
2022-07-25 18:19:08 72167a8923bc Coach[90] INFO NEW/PREV WINS : 21 / 9 ; DRAWS : 0
2022-07-25 18:19:08 72167a8923bc Coach[90] INFO ACC

Checkpoint Directory exists! 
Checkpoint Directory exists! 


Self Play:   2%|▏         | 3/200 [00:59<1:09:01, 21.02s/it]2022-07-25 18:20:42 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:20:42 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:   2%|▏         | 4/200 [01:36<1:29:42, 27.46s/it]2022-07-25 18:21:10 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:21:11 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:21:12 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:21:12 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:   2%|▎         | 5/200 [02:04<1:28:59, 27.38s/it]2022-07-25 18:21:47 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-25 18:21:51 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:   9%|▉         | 18/200 [08:37<1:

...

2022-07-26 11:18:28 72167a8923bc Coach[90] INFO Starting Iter #9 ...
Self Play:   2%|▏         | 4/200 [02:33<2:01:44, 37.27s/it]2022-07-26 11:21:39 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 11:21:39 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:   9%|▉         | 18/200 [08:25<1:00:20, 19.89s/it]2022-07-26 11:27:21 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 11:27:21 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:  20%|██        | 40/200 [18:28<1:25:03, 31.90s/it]2022-07-26 11:37:32 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 11:37:39 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Self Play:  21%|██        | 42/200 [19:45<1:31:25, 34.72s/it]2022-07-26 11:38:32 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-2

Checkpoint Directory exists! 
Epoch 1/10
10071/10071 [==============================] - 133s 13ms/step - loss: 2.0053 - pi_loss: 1.7205 - v_loss: 0.2848
Epoch 2/10
10071/10071 [==============================] - 133s 13ms/step - loss: 1.8673 - pi_loss: 1.6170 - v_loss: 0.2503
Epoch 3/10
10071/10071 [==============================] - 132s 13ms/step - loss: 1.8038 - pi_loss: 1.5676 - v_loss: 0.2362
Epoch 4/10
10071/10071 [==============================] - 132s 13ms/step - loss: 1.7604 - pi_loss: 1.5308 - v_loss: 0.2296
Epoch 5/10
10071/10071 [==============================] - 132s 13ms/step - loss: 1.7271 - pi_loss: 1.5037 - v_loss: 0.2234
Epoch 6/10
10071/10071 [==============================] - 132s 13ms/step - loss: 1.6999 - pi_loss: 1.4809 - v_loss: 0.2190
Epoch 7/10
10071/10071 [==============================] - 132s 13ms/step - loss: 1.6774 - pi_loss: 1.4625 - v_loss: 0.2148
Epoch 8/10
10071/10071 [==============================] - 131s 13ms/step - loss: 1.6570 - pi_loss: 1.4456 - v

2022-07-26 13:21:26 72167a8923bc Coach[90] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (1):  47%|████▋     | 7/15 [03:28<03:50, 28.83s/it]2022-07-26 13:25:25 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:25 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:25 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:25 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:26 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (1):  53%|█████▎    | 8/15 [03:59<03:28, 29.79s/it]2022-07-26 13:25:58 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:59 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 13:25:59 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-2

Checkpoint Directory exists! 
Epoch 1/10
10749/10749 [==============================] - 142s 13ms/step - loss: 2.2123 - pi_loss: 1.8780 - v_loss: 0.3343
Epoch 2/10
10749/10749 [==============================] - 140s 13ms/step - loss: 1.9948 - pi_loss: 1.7210 - v_loss: 0.2737
Epoch 3/10
10749/10749 [==============================] - 141s 13ms/step - loss: 1.8971 - pi_loss: 1.6435 - v_loss: 0.2536
Epoch 4/10
10749/10749 [==============================] - 140s 13ms/step - loss: 1.8328 - pi_loss: 1.5909 - v_loss: 0.2419
Epoch 5/10
10749/10749 [==============================] - 140s 13ms/step - loss: 1.7871 - pi_loss: 1.5533 - v_loss: 0.2338
Epoch 6/10
10749/10749 [==============================] - 141s 13ms/step - loss: 1.7518 - pi_loss: 1.5241 - v_loss: 0.2277
Epoch 7/10
10749/10749 [==============================] - 141s 13ms/step - loss: 1.7212 - pi_loss: 1.4989 - v_loss: 0.2223
Epoch 8/10
10749/10749 [==============================] - 140s 13ms/step - loss: 1.6978 - pi_loss: 1.4787 - v

2022-07-26 15:49:09 72167a8923bc Coach[90] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (1):  13%|█▎        | 2/15 [00:57<05:43, 26.40s/it]2022-07-26 15:50:22 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 15:50:48 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2):  27%|██▋       | 4/15 [02:21<06:04, 33.13s/it]2022-07-26 15:58:39 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
2022-07-26 15:58:43 72167a8923bc MCTS[90] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2): 100%|██████████| 15/15 [08:43<00:00, 34.91s/it]
2022-07-26 16:04:23 72167a8923bc Coach[90] INFO NEW/PREV WINS : 15 / 15 ; DRAWS : 0
2022-07-26 16:04:23 72167a8923bc Coach[90] INFO REJECTING NEW MODEL
2022-07-26 16:04:24 72167a8923bc Coach[90] INFO Starting Iter #11 ...
Self Play:  14%|█▎        | 27/200 [12:39<1:02:30, 21.68s/it]2022-07-26 16:17:24 72167a8923bc MCTS[90] ERR

In [ ]:
# Save progress by backing up the latest checkpoint, to be loaded on next run
latest_samples = sorted(checkpoint_folder.glob('*.examples'))[-1]
best_model = checkpoint_folder / 'best.h5'

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S') 

shutil.copy(latest_samples, load_folder / f'{timestamp}.pth.tar.examples')
shutil.copy(best_model, load_folder / f'{timestamp}.h5')

PosixPath('/content/gdrive/MyDrive/colab/alpha-zero-general/pretrained_models/santorini/keras/5x5/2022-07-27_12-49-53.h5')

In [ ]:
# Test performance against random bots

%matplotlib widget

game = SantoriniGame(5)

random_player = RandomPlayer(game).play
greedy_player = GreedySantoriniPlayer(game).play

args = dotdict({'numMCTSSims': 25, 'cpuct': 1.0})
mcts = MCTS(game, nnet, args)
alphago_player = lambda x: np.argmax(mcts.getActionProb(x, temp=0))

In [ ]:
# Play random player

arena = Arena.Arena(alphago_player, random_player, game, display=game.display)
%time oneWon, twoWon, draws = arena.playGames(200, verbose=False)
print("\AlphaGo won {} games, Random Player won {} games".format(oneWon, twoWon))

Arena.playGames (1):   0%|          | 0/100 [00:00<?, ?it/s]2022-07-27 12:54:11 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 12:54:22 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 12:54:27 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (1):  22%|██▏       | 22/100 [07:24<28:01, 21.56s/it]2022-07-27 13:01:26 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 13:01:26 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (1):  39%|███▉      | 39/100 [12:19<20:44, 20.41s/it]2022-07-27 13:06:21 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 13:06:21 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 13:06:22 e4b1754583f3 MCTS[88] ERROR All valid moves were masked, doing a workaround.
2022-07-27 13:06:22 e4b175

CPU times: user 58min 53s, sys: 4min 27s, total: 1h 3min 21s
Wall time: 1h 19s
\AlphaGo won 192 games, Random Player won 8 games


In [ ]:
# Play greedy player

arena = Arena.Arena(alphago_player, greedy_player, game, display=game.display)
%time oneWon, twoWon, draws = arena.playGames(200, verbose=False)
print("\AlphaGo won {} games, Greedy Player won {} games".format(oneWon, twoWon))

Arena.playGames (2): 100%|██████████| 100/100 [09:57<00:00,  5.98s/it]

CPU times: user 23min 26s, sys: 1min 29s, total: 24min 56s
Wall time: 23min 40s
\AlphaGo won 4 games, Greedy Player won 196 games


In [ ]:
# Compare random vs greedy player

arena = Arena.Arena(random_player, greedy_player, game, display=game.display)
%time oneWon, twoWon, draws = arena.playGames(00, verbose=False)
print("\Random won {} games, Greedy Player won {} games".format(oneWon, twoWon))

Arena.playGames (2): 100%|██████████| 250/250 [00:23<00:00, 10.43it/s]

CPU times: user 53.8 s, sys: 898 ms, total: 54.7 s
Wall time: 53.6 s
\Random won 0 games, Greedy Player won 500 games
